In [1]:
from random import random

from compas.colors import Color
from compas.geometry import Box, Sphere, Cylinder
from compas.geometry import Frame
from compas.geometry import Translation
from compas_viewer import Viewer
from compas.files import OBJ, STL
from compas.datastructures import Mesh


import numpy as np
from sklearn.manifold import Isomap
import matplotlib.pyplot as plt


In [3]:
mesh = Mesh.from_stl('model.stl')


mesh

In [ ]:
normalized_curvature = np.array([mesh.vertex_curvature(k) for k in mesh.vertices()])
normalized_curvature = normalized_curvature / np.max(np.abs(normalized_curvature))


vcol = {i: Color(-min(normalized_curvature[i], 0), 0, max(normalized_curvature[i], 0)) for i in mesh.vertices()}
vcol

{0: Color(red=0, green=0, blue=4.820115842455561e-05, alpha=1.0),
 1: Color(red=0, green=0, blue=4.820115842455528e-05, alpha=1.0),
 2: Color(red=0, green=0, blue=4.818882060877066e-05, alpha=1.0),
 3: Color(red=0.2945299343983292, green=0, blue=0, alpha=1.0),
 4: Color(red=0, green=0, blue=4.820352543060548e-05, alpha=1.0),
 5: Color(red=2.6161983664171566e-18, green=0, blue=0, alpha=1.0),
 6: Color(red=0, green=0, blue=4.8214519377454154e-05, alpha=1.0),
 7: Color(red=0, green=0, blue=4.820352543060548e-05, alpha=1.0),
 8: Color(red=0, green=0, blue=4.821451937745448e-05, alpha=1.0),
 9: Color(red=0, green=0, blue=4.82129525606757e-05, alpha=1.0),
 10: Color(red=0, green=0, blue=0.0017272774030794081, alpha=1.0),
 11: Color(red=0, green=0, blue=4.8152073958186115e-05, alpha=1.0),
 12: Color(red=0, green=0, blue=4.821295256067635e-05, alpha=1.0),
 13: Color(red=0, green=0, blue=4.8152073958186115e-05, alpha=1.0),
 14: Color(red=0, green=0, blue=4.816248551504099e-05, alpha=1.0),
 15: 

In [23]:
viewer = Viewer()
viewer.scene.clear()
#render mesh with vertex colors, faces, verticies, edges
obj1 = viewer.scene.add(mesh, use_vertexcolors=True, vertexcolor=vcol, show_faces=True, show_vertices=True, show_edges=True)
viewer.show()